# Imports

In [ ]:
from asapdiscovery.dataviz.plotting import scatter_plot
import pandas as pd
import os

In [ ]:
input_dir = "/Users/alexpayne/lilac-mount-point/asap-datasets/mers_fauxalysis/20230425_docked_for_fauxalysis/"

In [ ]:
df = pd.read_csv(os.path.join(input_dir, "best_results.csv"))

In [ ]:
sum(df.Docked_File.isna())

In [ ]:
sum(df.POSIT == -1)

# how many structures?

In [ ]:
structure_counts = df.groupby("Structure_Source")[["Structure_Source"]].count()

In [ ]:
structure_counts.columns = ["Counts"]

In [ ]:
import plotly.express as px

In [ ]:
fig = px.bar(structure_counts)

In [ ]:
fig.show()

In [ ]:
px.scatter(df, color="RMSD", x="POSIT", y="Chemgauss4", hover_data=["Complex_ID"], range_x=[0,1])

## What is the MEAN RMSD and posit score for the "best" molecules?

In [ ]:
by_compound = pd.read_csv(os.path.join(input_dir, "by_compound.csv"))

In [ ]:
by_compound_filtered = by_compound.loc[:, ["RMSD_Mean", "POSIT_R_Mean", "Compound_ID"]]

In [ ]:
by_compound_filtered["POSIT_R_Mean"] = 1-by_compound_filtered.POSIT_R_Mean

In [ ]:
by_compound_filtered.columns = ["RMSD_Mean", "POSIT_Mean", "Compound_ID"]

In [ ]:
by_compound_filtered

In [ ]:
import numpy as np
# df_extra = pd.merge(df, by_compound_filtered, on="Compound_ID")
df_extra = pd.merge(df, by_compound_filtered, on="Compound_ID").replace(-1, np.nan)

In [ ]:
px.scatter(df_extra, x="POSIT", y="POSIT_Mean", hover_data=["Compound_ID", "Structure_Source"], color="Structure_Source")

In [ ]:
fig = px.density_contour(df_extra, x="POSIT", y="POSIT_Mean")
fig.update_traces(contours_coloring="fill", contours_showlabels = True)
fig.show()

In [ ]:
len(df)

# Analyze v3

In [ ]:
input_dir = "/Users/alexpayne/lilac-mount-point/asap-datasets/mers_fauxalysis/20230425_docked_for_fauxalysis/csvs_v3/"

In [ ]:
df = pd.read_csv(os.path.join(input_dir, "best_results.csv"))

## Red Flags

In [ ]:
sum(df.Docked_File.isna())

In [ ]:
sum(df.POSIT == -1)

In [ ]:
sum(df.Clash == 1)

In [ ]:
df.loc[df.Clash == 1, :]

In [ ]:
df.Clash.unique()

# how many structures?

In [ ]:
structure_counts = df.groupby("Structure_Source")[["Structure_Source"]].count()

In [ ]:
structure_counts.columns = ["Counts"]

In [ ]:
import plotly.express as px

In [ ]:
fig = px.bar(structure_counts)

In [ ]:
fig.show()

In [ ]:
px.scatter(df, color="RMSD", x="POSIT", y="Chemgauss4", hover_data=["Complex_ID"], range_x=[0,1])

## What is the MEAN RMSD and posit score for the "best" molecules?

In [ ]:
by_compound = pd.read_csv(os.path.join(input_dir, "by_compound.csv"))

In [ ]:
by_compound_filtered = by_compound.loc[:, ["RMSD_Mean", "POSIT_R_Mean", "Compound_ID"]]

In [ ]:
by_compound_filtered["POSIT_R_Mean"] = 1-by_compound_filtered.POSIT_R_Mean

In [ ]:
by_compound_filtered.columns = ["RMSD_Mean", "POSIT_Mean", "Compound_ID"]

In [ ]:
by_compound_filtered

In [ ]:
import numpy as np
# df_extra = pd.merge(df, by_compound_filtered, on="Compound_ID")
df_extra = pd.merge(df, by_compound_filtered, on="Compound_ID").replace(-1, np.nan)

In [ ]:
px.scatter(df_extra, x="POSIT", y="POSIT_Mean", hover_data=["Compound_ID", "Structure_Source"], color="Structure_Source")

In [ ]:
fig = px.density_contour(df_extra, x="POSIT", y="POSIT_Mean")
fig.update_traces(contours_coloring="fill", contours_showlabels = True)
fig.show()

In [ ]:
len(df)

# How many clashes were there for the whole set?

In [ ]:
from asapdiscovery.docking.analysis import DockingResults

In [ ]:
dr = DockingResults(csv_path=os.path.join(input_dir, "all_results_cleaned.csv"))

In [ ]:
by_compound_clashes = dr.df.groupby("Compound_ID", group_keys=True)[["Clash"]].sum().rename(columns={"Clash": "Clash_Sum"})

In [ ]:
by_compound_clashes["Perc_Clash"] = by_compound_clashes.Clash_Sum / dr.df.groupby("Compound_ID", group_keys=True)[["Clash"]].count()["Clash"]

In [ ]:
by_compound_clashes.sort_values("Perc_Clash")

In [ ]:
px.bar(by_compound_clashes.sort_values(["Perc_Clash", "Compound_ID"]), y="Perc_Clash", color="Clash_Sum")

In [ ]:
best_df_plus_sum_clashes = pd.merge(df_extra, by_compound_clashes, on="Compound_ID")

In [ ]:
best_df_plus_sum_clashes

In [ ]:
best_df_plus_sum_clashes.loc[best_df_plus_sum_clashes.Clash == 1, ["Compound_ID", "Clash_Sum", "Perc_Clash"]]

In [ ]:
dr.df.loc[dr.df.Compound_ID == "MAT-POS-5d65ec79-2",:]